In [23]:
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
import sqlite3
import DataFunctions
import numpy as np


configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Data Preprocessing

Games DF

In [24]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"

# no longer need, right?
# try:
#     api_response = api_instance.get_games(year=year,division=division)
#     print(len(api_response))
# except:
#     print(ApiException)
    

In [25]:
gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=2022)
len(gamelist)

735

In [26]:
games_df = DataFunctions.df_from_games(gamelist=gamelist)
games_df.head()

,id,season,neutral_site,home_id,home_team,home_conference,home_points,away_id,away_team,away_conference,away_points,game_spread,game_totalpts
0,401405059,2022,True,77,Northwestern,Big Ten,31,158,Nebraska,Big Ten,28,3,59
1,401404146,2022,False,328,Utah State,Mountain West,31,41,Connecticut,FBS Independents,20,11,51
2,401405058,2022,False,356,Illinois,Big Ten,38,2751,Wyoming,Mountain West,6,32,44
3,401426530,2022,False,2226,Florida Atlantic,Conference USA,43,2429,Charlotte,Conference USA,13,30,56
4,401426531,2022,False,2638,UTEP,Conference USA,13,249,North Texas,Conference USA,31,-18,44


In [27]:
conn = sqlite3.connect("CollegeFootball.db")
games_df.to_sql("games",conn,if_exists="replace",index=False)
conn.close()


In [28]:
# we can see with this code that we can extract DF from SQL db

# conn = sqlite3.connect("CollegeFootball.db")
# trialdf = pd.read_sql_query("SELECT * FROM games",conn)
# conn.close()
# print(trialdf.shape)
# trialdf.head()


Betting lines DF

In [29]:
api_instance1 = cfbd.BettingApi(cfbd.ApiClient(configuration))

In [30]:
conferences=[]
for game in gamelist:
    conferences.append(game.away_conference)
conferences=set(conferences)

conferences

{'ACC',
 'American Athletic',
 'Big 12',
 'Big Ten',
 'Conference USA',
 'FBS Independents',
 'Mid-American',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt'}

In [31]:
betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=2022,conferences=conferences)

In [32]:
betting_df=DataFunctions.df_betting_lines(betting_list)
betting_df

#warning seems to be since betting line -0.0 for one of the games but still stores value as 0 so should be fine

c:\Users\rodri\anaconda3\envs\PIC16B\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\rodri\anaconda3\envs\PIC16B\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,id,av_spread,av_total
0,401411101,-24.375,51.375
1,401404079,7.125,65.625
2,401403893,1.500,51.000
3,401404080,2.500,45.000
4,401404081,13.625,53.000
...,...,...,...
733,401403949,-22.500,47.375
734,401403952,29.000,46.500
735,401404114,-2.000,58.125
736,401403953,-22.625,66.875


In [33]:
conn=sqlite3.connect("CollegeFootball.db")
betting_df.to_sql("betting_lines",conn,if_exists="replace",index=False)
conn.close()

In [34]:
#sample merge dataframe

cmd=\
"""
SELECT G.away_team,G.away_points,G.home_points,G.home_team,B.av_spread,B.av_total
FROM games G
INNER JOIN betting_lines B ON G.id=B.id
"""
conn=sqlite3.connect("CollegeFootball.db")
test_merge=pd.read_sql_query(cmd,conn)
conn.close() # do not forget this after query
test_merge

,away_team,away_points,home_points,home_team,av_spread,av_total
0,Nebraska,28,31,Northwestern,-12.375,52.250
1,Connecticut,20,31,Utah State,23.750,59.250
2,Wyoming,6,38,Illinois,14.000,42.750
3,Charlotte,13,43,Florida Atlantic,7.000,60.000
4,North Texas,31,13,UTEP,-1.375,52.875
...,...,...,...,...,...,...
730,LSU,30,50,Georgia,17.125,51.750
731,Fresno State,28,16,Boise State,3.000,54.750
732,Clemson,39,10,North Carolina,-7.250,64.000
733,Purdue,22,43,Michigan,15.625,53.750


Team stats per season DF

In [35]:
api_instance2 = cfbd.StatsApi(cfbd.ApiClient(configuration))

In [36]:
teamstats = api_instance2.get_advanced_team_season_stats(year=year)

In [37]:
len(teamstats)

131

In [38]:
teamstats = [{**{'team':t.team,'season':t.season,'conference':t.conference},
                **DataFunctions.word_adder(word="Offensive",dict=t.offense.to_dict()),
                **DataFunctions.word_adder(word="Defensive",dict=t.defense.to_dict())} for t in teamstats]


In [48]:
teamstat0 = teamstats[0]
type(teamstat0)

dict

In [50]:
updaterkeys = set()
for key,value in teamstat0.items():
    if type(value)==type(teamstat0):
        updaterkeys.add(key)
        
updaterkeys


{'Defensive field_position',
 'Defensive havoc',
 'Defensive passing_downs',
 'Defensive passing_plays',
 'Defensive rushing_plays',
 'Defensive standard_downs',
 'Offensive field_position',
 'Offensive havoc',
 'Offensive passing_downs',
 'Offensive passing_plays',
 'Offensive rushing_plays',
 'Offensive standard_downs'}

Merging tables to set up for NN